# Analysis DATA

In [ ]:
from utilities import *

raw_data = pd.read_excel("data/Datos_Market_copy.xlsx")

sales_analysis = SalesAnalysis(raw_data)

data = sales_analysis.data

### Create our model

In [ ]:
data_dummies, model = sales_analysis.modelization(data[sales_analysis.brand35])

model.summary()

#### Plot the residuals

In [ ]:
sales_analysis.plot_resid_ACF_PACF(model)